### Homework

1. Load in a generative model using the HuggingFace pipeline. Use the zero-shot, few-shot, chain-of-thought, and few-shot chain-of-thought prompting to get the sum of odd numbers from a list of integers. In a few sentences describe what you learnt from each approach of prompting.
- Next, play around with the temperature parameter. In a few sentences describe what you changes you notice.

In [31]:
import GPUtil

GPUs = GPUtil.getGPUs()
for gpu in GPUs:
    print(f"GPU {gpu.id}: {gpu.memoryUsed} / {gpu.memoryTotal} MB")

GPU 0: 40230.0 / 40960.0 MB
GPU 1: 15001.0 / 40960.0 MB


In [21]:
import torch, gc



torch.cuda.set_device(0)
gc.collect()
torch.cuda.empty_cache()

with torch.no_grad():
    torch.cuda.empty_cache()

torch.cuda.set_device(1)
gc.collect()
torch.cuda.empty_cache()

with torch.no_grad():
    torch.cuda.empty_cache()

In [25]:
for var_name in dir():
    var = eval(var_name)
    if torch.is_tensor(var) and var.is_cuda:
        del var

del model

In [18]:
torch.cuda

<module 'torch.cuda' from '/soft/applications/conda/2024-08-08/mconda3/lib/python3.11/site-packages/torch/cuda/__init__.py'>

In [5]:
import torch, torchvision
device = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

In [8]:
!nvidia-smi

Tue Nov 26 18:28:17 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:07:00.0 Off |                    0 |
| N/A   22C    P0             56W /  400W |   40230MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [1]:
import os
from getpass import getpass
os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass('Enter huggingfacehub api token: ')

Enter huggingfacehub api token:  ········


In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

# load model
model_name = "microsoft/Phi-3-mini-4k-instruct" #"tiiuae/falcon-7b-instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir='../')
# model = "tiiuae/falcon-40b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipeline = transformers.pipeline("text-generation",
                                        model=model,
                                        tokenizer=tokenizer,
                                        torch_dtype=torch.bfloat16,
                                        trust_remote_code=True,
                                        device_map="auto",
                                        device=device
                                        )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Both `device` and `device_map` are specified. `device` will override `device_map`. You will most likely encounter unexpected behavior. Please remove `device` and keep `device_map`.


In [96]:
# load remote model
from langchain.llms import HuggingFaceHub
# model = "tiiuae/falcon-7b-instruct"
# model = "tiiuae/falcon-40b-instruct"
remote_model = HuggingFaceHub(
    repo_id=model_name,
    model_kwargs={"temperature": 0.5,
                  "max_new_tokens": 2056},
)

In [91]:
def get_completion(input, host="remote", **kwargs):
    prompt = f"#### User: \n{input}\n\n#### Response from model:"
    response = ""
    # print(prompt)
    if host.lower() == "local":
      print("invoking llm at Google Colab")
      if 'max_length' not in kwargs:
        kwargs['max_length'] = 1000

      
        _response = pipeline(prompt,
                          #max_length=500,
                          do_sample=True,
                          top_k=10,
                          num_return_sequences=1,
                          eos_token_id=tokenizer.eos_token_id,
                          **kwargs,
                          )
      response = _response[0]['generated_text']

    elif host.lower() == "remote":
      print("invoking llm at Huggingface Hub")
      if "max_length" in kwargs:
        kwargs['max_new_tokens'] = kwargs['max_length']

      response = remote_model.invoke(prompt, **kwargs)

    else:
      print ("invalid host value, must be 'remote' or 'local'")

    return response

# Zero-shot prompting

In [92]:
zero_shot_prompt =  """
Problem:
Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:
"""
#15 + 5 + 13 + 7 + 1 = 41
zero_shot_prompt_explanation = get_completion(zero_shot_prompt)
print(zero_shot_prompt_explanation)

invoking llm at Huggingface Hub
#### User: 

Problem:
Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:


#### Response from model:

Let's identify the odd numbers in the given group:

15, 5, 13, 7, 1

Now, let's add them up:

15 + 5 + 13 + 7 + 1 = 41

The sum of the odd numbers is 41, which is an odd number, not an even number.






The model was right.

# Few-shot prompting

In [97]:
few_shot_prompt =  """I will provide you with a few examples of a math problem.
Follow the pattern to solve a problem.
Example1:
Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: False

Example2:
Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: True.

Now solve the following problem. Explain the steps involved.
Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:
"""
#15 + 5 + 13 + 7 + 1 = 41
few_shot_prompt_explanation = get_completion(few_shot_prompt)
print(few_shot_prompt_explanation)

invoking llm at Huggingface Hub
#### User: 
I will provide you with a few examples of a math problem.
Follow the pattern to solve a problem.
Example1:
Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: False

Example2:
Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: True.

Now solve the following problem. Explain the steps involved.
Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:


#### Response from model:

To determine if the odd numbers in the group add up to an even number, we first need to identify all the odd numbers in the group. Then, we add them together and check if the sum is even.

Step 1: Identify the odd numbers in the group.
Odd numbers: 15, 5, 13, 7, 1

Step 2: Add the odd numbers together.
Sum = 15 + 5 + 13 + 7 + 1 = 41

Step 3: Check if the sum is even.
Since 41 is not divisible by 2, it is an odd number.

Conclusion: The odd numbers in the group do not add

Again right

# Few-shot COT

In [94]:
few_shot_COT_prompt =  """You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25. 25 is an odd number, hence the answer is False.

Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36. 36 is an even number, hence the answer is True.

Q: The odd numbers in this group add up to an even number: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding all the selected odd numbers (11, 13) gives 24. 24 is an even number, hence the answer is True.

Q: The odd numbers in this group add up to an even number: 17, 9, 10, 12, 13, 4, 2.
A: The given numbers are: (17, 9, 10, 12, 13, 4, 2). Next select the odd numbers (17, 9, 13).  Adding all the selected odd numbers (17, 9, 13) gives 39. 39 is an odd number, hence the answer is False.

Problem:
Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:
"""
#15 + 5 + 13 + 7 + 1 = 41
few_shot_COT_prompt_explanation = get_completion(few_shot_COT_prompt)
print(few_shot_COT_prompt_explanation)

invoking llm at Huggingface Hub
#### User: 
You are a mathematical genius.
The template of the question is given as Q: [question]. The template of the answer is given as A: [answer].
Help me solve the problem below using the examples given.

Examples:
Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The given numbers are: (4, 8, 9, 15, 12, 2, 1). Next select the odd numbers (9, 15, 1).  Adding all the selected odd numbers (9, 15, 1) gives 25. 25 is an odd number, hence the answer is False.

Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: The given numbers are: (17, 10, 19, 4, 8, 12, 24). Next select the odd numbers (17, 19).  Adding all the selected odd numbers (17, 19) gives 36. 36 is an even number, hence the answer is True.

Q: The odd numbers in this group add up to an even number: 16, 11, 14, 4, 8, 13, 24.
A: The given numbers are: (16, 11, 14, 4, 8, 13, 24). Next select the odd numbers (11, 13).  Adding a

The model is correct.

# Few-shot prompting (varying temperature)

In [100]:
few_shot_prompt =  """I will provide you with a few examples of a math problem.
Follow the pattern to solve a problem.
Example1:
Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: False

Example2:
Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: True.

Now solve the following problem. Explain the steps involved.
Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:
"""


remote_model = HuggingFaceHub(
    repo_id=model_name,
    model_kwargs={"temperature": 0.9,
                  "max_new_tokens": 2056},
)

def get_completion(input, host="remote", **kwargs):
    prompt = f"#### User: \n{input}\n\n#### Response from model:"
    response = ""
    # print(prompt)
    if host.lower() == "local":
      print("invoking llm at Google Colab")
      if 'max_length' not in kwargs:
        kwargs['max_length'] = 1000

      
        _response = pipeline(prompt,
                          #max_length=500,
                          do_sample=True,
                          top_k=10,
                          num_return_sequences=1,
                          eos_token_id=tokenizer.eos_token_id,
                          **kwargs,
                          )
      response = _response[0]['generated_text']

    elif host.lower() == "remote":
      print("invoking llm at Huggingface Hub")
      if "max_length" in kwargs:
        kwargs['max_new_tokens'] = kwargs['max_length']

      response = remote_model.invoke(prompt, **kwargs)

    else:
      print ("invalid host value, must be 'remote' or 'local'")

    return response

few_shot_prompt_explanation = get_completion(few_shot_prompt)
print(few_shot_prompt_explanation)

invoking llm at Huggingface Hub
#### User: 
I will provide you with a few examples of a math problem.
Follow the pattern to solve a problem.
Example1:
Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: False

Example2:
Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: True.

Now solve the following problem. Explain the steps involved.
Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:


#### Response from model: False
To solve this problem, we need to find the odd numbers in the given group and then add them together. The odd numbers in the group are 15, 5, 13, and 7. Adding these numbers together gives us:

15 + 5 + 13 + 7 = 40

Since 40 is an even number, the statement is True.





In [101]:
remote_model = HuggingFaceHub(
    repo_id=model_name,
    model_kwargs={"temperature": 1,
                  "max_new_tokens": 2056},
)

def get_completion(input, host="remote", **kwargs):
    prompt = f"#### User: \n{input}\n\n#### Response from model:"
    response = ""
    # print(prompt)
    if host.lower() == "local":
      print("invoking llm at Google Colab")
      if 'max_length' not in kwargs:
        kwargs['max_length'] = 1000

      
        _response = pipeline(prompt,
                          #max_length=500,
                          do_sample=True,
                          top_k=10,
                          num_return_sequences=1,
                          eos_token_id=tokenizer.eos_token_id,
                          **kwargs,
                          )
      response = _response[0]['generated_text']

    elif host.lower() == "remote":
      print("invoking llm at Huggingface Hub")
      if "max_length" in kwargs:
        kwargs['max_new_tokens'] = kwargs['max_length']

      response = remote_model.invoke(prompt, **kwargs)

    else:
      print ("invalid host value, must be 'remote' or 'local'")

    return response

few_shot_prompt_explanation = get_completion(few_shot_prompt)
print(few_shot_prompt_explanation)

invoking llm at Huggingface Hub
#### User: 
I will provide you with a few examples of a math problem.
Follow the pattern to solve a problem.
Example1:
Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: False

Example2:
Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: True.

Now solve the following problem. Explain the steps involved.
Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:


#### Response from model:

To solve this problem, we need to identify the odd numbers in the given group and then add them up to see if the sum is an even number.

Step 1: Identify the odd numbers in the group.
Odd numbers are those that have a remainder of 1 when divided by 2. In the given group, the odd numbers are: 15, 5, 13, 7, and 1.

Step 2: Add the odd numbers together.
15 + 5 + 13 + 7 + 1 = 41

Step 3: Determine if the sum is an even number.
41 is an odd number, as it has a remainder of 1 

In [102]:
remote_model = HuggingFaceHub(
    repo_id=model_name,
    model_kwargs={"temperature": 0.3,
                  "max_new_tokens": 2056},
)

def get_completion(input, host="remote", **kwargs):
    prompt = f"#### User: \n{input}\n\n#### Response from model:"
    response = ""
    # print(prompt)
    if host.lower() == "local":
      print("invoking llm at Google Colab")
      if 'max_length' not in kwargs:
        kwargs['max_length'] = 1000

      
        _response = pipeline(prompt,
                          #max_length=500,
                          do_sample=True,
                          top_k=10,
                          num_return_sequences=1,
                          eos_token_id=tokenizer.eos_token_id,
                          **kwargs,
                          )
      response = _response[0]['generated_text']

    elif host.lower() == "remote":
      print("invoking llm at Huggingface Hub")
      if "max_length" in kwargs:
        kwargs['max_new_tokens'] = kwargs['max_length']

      response = remote_model.invoke(prompt, **kwargs)

    else:
      print ("invalid host value, must be 'remote' or 'local'")

    return response

few_shot_prompt_explanation = get_completion(few_shot_prompt)
print(few_shot_prompt_explanation)

invoking llm at Huggingface Hub
#### User: 
I will provide you with a few examples of a math problem.
Follow the pattern to solve a problem.
Example1:
Q: The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: False

Example2:
Q: The odd numbers in this group add up to an even number: 17, 10, 19, 4, 8, 12, 24.
A: True.

Now solve the following problem. Explain the steps involved.
Q: The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
A:


#### Response from model:

To solve the problem, we need to identify the odd numbers in the group and then add them up to see if the sum is even.

Step 1: Identify the odd numbers in the group.
Odd numbers are those that have a remainder of 1 when divided by 2. In the given group, the odd numbers are: 15, 5, 13, 7, and 1.

Step 2: Add up the identified odd numbers.
15 + 5 + 13 + 7 + 1 = 41

Step 3: Determine if the sum is even or odd.
41 is an odd number because it has a remainder of 1 when divide